In [1]:
import pandas as pd

In [2]:
# Fetch the CSET data to cross reference with your data.
cset_data = pd.read_csv("cset_data.csv", index_col="date")

In [3]:
csv_files = [
    "../data/chinese_imports_of_SME_2022/comtrade2006-2010.csv",
    "../data/chinese_imports_of_SME_2022/comtrade_2011-2015.csv",
    "../data/chinese_imports_of_SME_2022/comtrade_2016-2020.csv"
]

In [4]:
dfs = []
for file in csv_files:
    df = pd.read_csv(file)
    df = (
        df
        .rename(columns={"Trade Value (US$)": "quant", "Year": "date"})
        .loc[df["Partner"]!="World",["date", "quant", "Partner"]]
        .groupby(["date", "Partner"])["quant"]
        .aggregate(sum)
        .reset_index(level="Partner", drop=False)
        )
    dfs.append(df)
my_data = pd.concat(dfs, axis=0, ignore_index=False)

Check numbers against CSET's

In [5]:
my_data_agg = my_data.groupby("date").aggregate(sum)

my_data_agg["Partner"] = "World"

compare = my_data_agg.merge(cset_data, left_index=True, right_index=True, suffixes=["_mine", "_csets"], how="inner")

(compare["quant_mine"] - compare["quant_csets"] == 0).all()

True

Combine aggregated with partner specific (only include major partners).

In [6]:
my_data = my_data.loc[my_data["quant"] > 1e9]

my_data = pd.concat([my_data, my_data_agg], axis=0, ignore_index=False)

my_data.to_csv("data_cleaned.csv")